In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen, Request
import re

Задаем DataFrame

In [2]:
df = pd.DataFrame(
    {"фрукт": ["яблоко" , "яблоко", "яблоко", "виноград", "виноград"],
    "сорт": ["голден", "зеленое", "раннее", "белый", "киш-миш"],
    "цена": [3, 200, 230, 5, 350],
    "валюта": ["евро", "руб", "руб", "евро", "руб"]})
df

,фрукт,сорт,цена,валюта
0,яблоко,голден,3,евро
1,яблоко,зеленое,200,руб
2,яблоко,раннее,230,руб
3,виноград,белый,5,евро
4,виноград,киш-миш,350,руб


Парсим текущий курс евро.

In [5]:
url = 'https://www.banki.ru/products/currency/'
html = urlopen(url)
soup = BeautifulSoup(html)
text = soup.find_all("tbody")[0].text
eur = float(re.findall(r".+", text)[4].replace(",","."))
eur

86.2195

Создаем столбец сопоставимых цен и заносим в него значения.

In [61]:
df.loc[df.валюта == "евро", "цена_конвертированная"] = df[df.валюта == "евро"]["цена"] * eur
df.loc[df.валюта != "евро", "цена_конвертированная"] = df[df.валюта != "евро"]["цена"]
df

,фрукт,сорт,цена,валюта,цена_конвертированная
0,яблоко,голден,3,евро,265.299
1,яблоко,зеленое,200,руб,200.000
2,яблоко,раннее,230,руб,230.000
3,виноград,белый,5,евро,442.165
4,виноград,киш-миш,350,руб,350.000


Выведем самый дорогой фрукт и его цену.

In [71]:
expensive = df[df.цена_конвертированная == np.max(df.цена_конвертированная)]
expensive

,фрукт,сорт,цена,валюта,цена_конвертированная
3,виноград,белый,5,евро,442.165


In [92]:
print(f"Самый дорогой фрукт - {expensive.фрукт.iloc[0]}, его цена "
      f"составляет {np.round(expensive.цена_конвертированная.iloc[0], 2)} рублей.")

Самый дорогой фрукт - виноград, его цена составляет 442.16 рублей.
